In [ ]:
from google.colab import files
files.upload()



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ratnalikarpranav","key":"8e1aa538e67d10cd4c1450417cd683c0"}'}

In [ ]:
!mkdir -p /root/.config/kaggle
!mv kaggle.json /root/.config/kaggle/
!pip install tensorflow_addons

In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-addons

  Using cached tensorflow-2.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.9.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached ml_dtypes-0.5.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (21 kB)
Using cached tensorflow-2.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (644.9 MB)
Using cached keras-3.9.2-py3-none-any.whl (1.3 MB)
Using cached ml_dtypes-0.5.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.7 MB)
Using cached tensorboard-2.19.0-py3-none-any.whl (5.5 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Atte

In [ ]:
!pip uninstall tensorflow keras -y
!pip install tensorflow==2.15.0


Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
Found existing installation: keras 3.9.2
Uninstalling keras-3.9.2:
  Successfully uninstalled keras-3.9.2
  Using cached tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
  Using cached ml_dtypes-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached tensorboard-2.15.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
Using cached tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (475.3 MB)
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
Using cached ml_dtypes-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
Using cached tensorboard-2.15.2-py3-none-any.whl (5.5 MB)
  Attempting uninstall: ml-dtypes
    Found existing installation: ml_dtypes 0.5.1
    Uninstalling ml_dtypes-0.5

In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import kaggle
import zipfile
import cv2
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

/usr/local/lib/python3.11/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

def focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * tf.math.log(y_pred)
        loss = alpha * tf.pow(1. - y_pred, gamma) * cross_entropy
        return tf.reduce_mean(tf.reduce_sum(loss, axis=-1))
    return loss

def preprocess_image(img):
    """Enhanced preprocessing pipeline"""
    # Adaptive thresholding
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY_INV, 11, 2)
    # Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    # Skeletonization
    skeleton = np.zeros(img.shape, np.uint8)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
    while cv2.countNonZero(img) > 0:
        eroded = cv2.erode(img, element)
        temp = cv2.dilate(eroded, element)
        skeleton = cv2.bitwise_or(skeleton, cv2.subtract(img, temp))
        img = eroded.copy()
    return skeleton.astype(np.float32) / 255.0

def load_data(data_dir):
    """Data loading function"""
    train_dir = os.path.join(data_dir, 'dhcd', 'train')
    test_dir = os.path.join(data_dir, 'dhcd', 'test')

    X_train, y_train = [], []
    X_test, y_test = [], []
    class_mapping = {}

    print("Loading training data...")
    classes = sorted(os.listdir(train_dir))
    for class_idx, class_name in enumerate(classes):
        class_mapping[class_idx] = class_name
        class_path = os.path.join(train_dir, class_name)
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (32, 32))
            img = preprocess_image(img)
            X_train.append(img)
            y_train.append(class_idx)

    print("Loading test data...")
    for class_idx, class_name in enumerate(classes):
        class_path = os.path.join(test_dir, class_name)
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (32, 32))
            img = preprocess_image(img)
            X_test.append(img)
            y_test.append(class_idx)

    # Convert to numpy arrays
    X_train = np.expand_dims(np.array(X_train), axis=-1)
    X_test = np.expand_dims(np.array(X_test), axis=-1)
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    # One-hot encoding
    num_classes = len(classes)
    y_train_cat = to_categorical(y_train, num_classes)
    y_test_cat = to_categorical(y_test, num_classes)

    return X_train, y_train_cat, X_test, y_test_cat, y_train, y_test, class_mapping

def build_model(input_shape, num_classes):
    """Enhanced CNN architecture"""
    model = Sequential([
        Conv2D(32, (3,3), padding='same', activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(32, (3,3), padding='same', activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.3),

        Conv2D(64, (3,3), padding='same', activation='relu'),
        BatchNormalization(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.4),

        Conv2D(128, (3,3), padding='same', activation='relu'),
        BatchNormalization(),
        Conv2D(128, (3,3), padding='same', activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.5),

        Flatten(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                loss=focal_loss(gamma=2, alpha=0.25),
                metrics=['accuracy'])
    return model

def main():
    # Dataset setup
    dataset_name = "ashokpant/devanagari-character-dataset-large"
    data_dir = "devanagari_data"

    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        kaggle.api.dataset_download_files(dataset_name, path=data_dir)
        for zip_file in [f for f in os.listdir(data_dir) if f.endswith('.zip')]:
            with zipfile.ZipFile(os.path.join(data_dir, zip_file), 'r') as zip_ref:
                zip_ref.extractall(data_dir)

    # Load data
    X_train, y_train_cat, X_test, y_test_cat, y_train, y_test, class_mapping = load_data(data_dir)

    # Train/val split
    X_train, X_val, y_train_cat, y_val_cat = train_test_split(
        X_train, y_train_cat, test_size=0.1, random_state=42)

    # Class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights_dict = dict(enumerate(class_weights))

    # Data augmentation
    train_datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        shear_range=0.2,
        brightness_range=[0.7,1.3],
        fill_mode='nearest',
        preprocessing_function=lambda x: x + tf.random.normal(x.shape, stddev=0.1)
    )

    # Build model
    model = build_model(X_train[0].shape, len(class_mapping))
    model.summary()

    # Callbacks
    callbacks = [
        EarlyStopping(patience=15, restore_best_weights=True),
        ReduceLROnPlateau(factor=0.2, patience=5),
        ModelCheckpoint('best_model.h5', save_best_only=True)
    ]

    # Training
    history = model.fit(
        train_datagen.flow(X_train, y_train_cat, batch_size=128),
        epochs=50,
        validation_data=(X_val, y_val_cat),
        callbacks=callbacks,
        class_weight=class_weights_dict,
        verbose=1
    )

    # Evaluation
    test_loss, test_acc = model.evaluate(X_test, y_test_cat)
    print(f"\nTest Accuracy: {test_acc:.4f}")

    # Classification report
    y_pred = model.predict(X_test)
    print("\nClassification Report:")
    print(classification_report(y_test, np.argmax(y_pred, axis=1),
                               target_names=list(class_mapping.values())))

if __name__ == "__main__":
    main()

Loading training data...
Loading test data...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 conv2d_2 

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/50
550/550 [==============================] - 659s 1s/step - loss: 0.9297 - accuracy: 0.0365 - val_loss: 0.8412 - val_accuracy: 0.1004 - lr: 0.0010
Epoch 3/50
550/550 [==============================] - 651s 1s/step - loss: 0.9124 - accuracy: 0.0424 - val_loss: 0.7447 - val_accuracy: 0.1552 - lr: 0.0010
Epoch 4/50
181/550 [========>.....................] - ETA: 22:45 - loss: 0.8960 - accuracy: 0.0459